In [ ]:
import os
os.getcwd()
os.chdir("..")

In [ ]:
os.getcwd()


In [ ]:
# Optimizing the chucking process
import os
import pdfplumber
import re


# python PDFProcessing/PDFProccesing.py > outputs/refoutput.txt

text_list = []

with pdfplumber.open("PDF_docs/doc_0.pdf") as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if text:
            text_list.append(text)
            text = text.join(text_list)
text





In [ ]:
parts = re.split(r"\breferences\b", text, flags=re.IGNORECASE, maxsplit=1)

body = parts[0].strip()  # Everything before "References"
reference = parts[1].strip() if len(parts) > 1 else ""  # Everything after "References"



In [ ]:
reference

In [ ]:
# Get meta data

In [ ]:
pdf = pdfplumber.open("PDF_docs/doc_2.pdf")
pdf.metadata

In [ ]:
pdf = pdfplumber.open("PDF_docs/doc_0.pdf")
metadata = pdf.metadata  # Extrahiere Metadaten
if metadata and "Title" in metadata and metadata["Title"]:
    print( f"Titel: {metadata['Title']}")
else: 
    print(False)

In [ ]:
with pdfplumber.open("PDF_docs/doc_1.pdf") as pdf:
    metadata = pdf.metadata
    print(metadata)

In [1]:
from ollama import ChatResponse, chat
from langchain_ollama import OllamaEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_chroma import Chroma
import chromadb

In [2]:
import os
os.chdir("..")

In [ ]:
pdf_dir = "PDF_docs/"

pdf_files = os.listdir(pdf_dir)

# Liste zum Speichern aller geladenen Dokumente
all_docs = []

# Alle PDFs im Verzeichnis durchgehen und laden
for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_dir, pdf_file)
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()
    all_docs.extend(docs)  # Alle Dokumente speichern

# Überprüfen, ob Dokumente geladen wurden
print(f"{len(all_docs)} Dokumente geladen.")


In [ ]:
[len(all_docs)]

In [ ]:
embeddings = OllamaEmbeddings(model="llama3.2:latest")  # Korrekte Embeddings-Funktion
embeddings

In [ ]:
import chromadb
persistent_client = chromadb.PersistentClient()
collection = persistent_client.get_or_create_collection("collection_name")

collection.add(ids=[len(all_docs)], documents=["a", "b", "c"])

vector_store_from_client = Chroma(
    client=persistent_client,
    collection_name="collection_name",
    embedding_function=embeddings,
)

In [ ]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
    id=1,
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
    id=2,
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
    id=3,
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
    id=4,
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
    id=5,
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
    id=6,
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
    id=7,
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
    id=8,
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
    id=9,
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
    id=10,
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_ollama import OllamaEmbeddings
import chromadb
from langchain_chroma import Chroma

# # 📌 Schritt 1: PDFs laden
# pdf_dir = "PDF_docs/"
# pdf_files = [f for f in os.listdir(pdf_dir) if f.endswith(".pdf")]  # Nur PDFs

# all_docs = []
# for pdf_file in pdf_files:
#     pdf_path = os.path.join(pdf_dir, pdf_file)
#     loader = PyPDFLoader(pdf_path)
#     docs = loader.load()
#     all_docs.extend(docs)  # Dokumente speichern

# print(f"✅ {len(all_docs)} Dokumente geladen.")

# # 📌 Schritt 2: Embeddings erstellen
# embeddings = OllamaEmbeddings(model="llama3.2:latest")  # Korrekte Embeddings

# # 📌 Schritt 3: ChromaDB persistent speichern
# persistent_client = chromadb.PersistentClient(path="./chroma_langchain_db")  # Verzeichnis für Speicherung
# collection = persistent_client.get_or_create_collection("collection_name")

# 📌 Schritt 4: Dokumente zu ChromaDB hinzufügen
doc_texts = [doc.page_content for doc in all_docs]  # Extrahiere den Text
doc_ids = [f"doc_{i}" for i in range(len(doc_texts))]  # Einzigartige IDs

collection.add(ids=doc_ids, documents=doc_texts)  # Speichern in ChromaDB

# 📌 Schritt 5: Chroma-VectorStore mit gespeicherten Daten initialisieren
vector_store_from_client = Chroma(
    persist_directory="./chroma_langchain_db",
    collection_name="collection_name",
    embedding_function=embeddings,
)

print("✅ ChromaDB erfolgreich mit PDFs initialisiert!")


In [ ]:
file_path = "PDF_docs/"

def save_doc(file_path):
    # Schritt 1: PDFs laden
    file_path = "PDF_docs/"
    pdf_files = [f for f in os.listdir(file_path) if f.endswith(".pdf")]  # Nur PDFs

    all_docs = []
    for pdf_file in pdf_files:
        pdf_path = os.path.join(file_path, pdf_file)
        loader = PyPDFLoader(pdf_path)
        docs = loader.load()
        all_docs.extend(docs)  # Dokumente speichern
    
    return all_docs

In [4]:
all_docs = save_doc(file_path=file_path)
len(all_docs)

117

In [5]:
def embedding():
    embeddings = OllamaEmbeddings(model="llama3.2:latest")  # Korrekte Embeddings
    return embeddings

In [6]:
embeddings = embedding() 
embeddings

OllamaEmbeddings(model='llama3.2:latest', base_url=None, client_kwargs={})

In [13]:
def persistent_clientChroma():
    persistent_client = chromadb.PersistentClient(path="./chroma_langchain_db")  # Verzeichnis für Speicherung
    collection = persistent_client.get_or_create_collection("collection_name")
    return collection

collection = persistent_clientChroma()
collection

Collection(name=collection_name)

In [14]:
def add_doc_to_Chroma():
    doc_texts = [doc.page_content for doc in all_docs]  # Extrahiere den Text
    doc_ids = [f"doc_{i}" for i in range(len(doc_texts))]  # Einzigartige IDs

    collection_db = collection.add(ids=doc_ids, documents=doc_texts)  # Speichern in ChromaDB
    return collection_db

In [15]:
collection_db = add_doc_to_Chroma()
collection_db

In [ ]:
def vector_store_from_clientChroma():
    # 📌 Schritt 5: Chroma-VectorStore mit gespeicherten Daten initialisieren
    vector_store_from_client = Chroma(
        persist_directory="./chroma_langchain_db",
        collection_name="collection_name",
        embedding_function=embeddings,
    )
